In [1]:
import pandas as pd
import sys
import os
sys.path.append("/Users/marc/Desktop/PostDoc/Mutation/signature5/notebooks")
import tools

col_order = ['Chr', 'Pos', 'Ref', 'Alt', 'Sample', 'Tissue_CellType', 'Compartment',
             'Individual', 'Study', 'Context', 'GeneralType', 'Type']
col_meta = ["Study", "Sample", "Individual", "Age", "Compartment", "Status", "Tissue_CellType"]

Mutation matrix for SigProfiler

In [46]:
mat = pd.read_csv("small_bowel/data/mutation_matrices/sbs_small_bowel_persample.txt", sep="\t")
mat.columns = [f"{c[4]}[{c[0:3]}]{c[-1]}" for c in mat.columns]
example = pd.read_csv("../Ng2021/Ng2021-signet.tab", sep="\t")

# Recorder to match signet output
mat = mat[[c for c in example.columns if "[" in c]].reset_index()
mat = mat.rename(columns = {"index":"Sample"})

mat.to_csv("Wang2023-signet.onlyspec.tab")

Read data

In [2]:
dat = pd.read_csv("substitutions_merged.tsv.gz",sep="\t")
meta = pd.read_excel("41588_2022_1296_MOESM4_ESM.xlsx", sheet_name="4_crypt_summary", skiprows=1)
ages = meta.set_index("patient")["age"].to_dict()
tissue = meta.set_index("patient")["region"].to_dict()
stat = meta.set_index("patient")["condition"].to_dict()

In [14]:
dat_l = []
with open(f"small_bowel/data/vcf/{vcf}", "r") as fh:
    sample = vcf.replace("_final_snp.vcf", "")
    for line in fh:
        if line.startswith("#"): continue
        line = line.strip()
        fields = line.split()
        dat_l.append([fields[0], fields[1], fields[3], fields[4], sample])
dat0 = pd.DataFrame(dat_l)
dat0.columns = ["Chr", "Pos", "Ref", "Alt", "Sample"]

Add missing columns

In [ ]:
dat["Individual"] = dat["Patient"]
dat["Study"] = "Wang2023"
dat["Compartment"] = "WGS"
dat["Chr"] = "chr" + dat["Chr"]
dat["Tissue_CellType"] = "Small intestine"

In [4]:
dat = tools.keep_snps(dat).reset_index(drop="True")
dat = dat[~dat["Chr"].isin(["chrX", "chrY"])]

In [5]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg19.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [6]:
dat[col_order].to_csv("Wang2023.csv.gz", compression="gzip", index=False)

Metafile

In [6]:
pmeta = pd.read_excel("41588_2022_1296_MOESM4_ESM.xlsx", sheet_name="4_crypt_summary", skiprows=1)

In [10]:
l = list(dat.Sample.unique())
meta = pd.DataFrame(l)
meta.columns = ["Sample"]
meta["Study"] = "Wang2023"
meta["Individual"] = [i.split("_")[0] for i in meta.Sample]
#meta["Tissue_CellType"] = "meta["Individual"].map(tissue)"
meta["Tissue_CellType"] = "Small intestine"
meta["Age"] = meta["Individual"].map(ages)
meta["Status"] = meta["Individual"].map(stat)
meta["Compartment"] = "WGS"

In [13]:
meta[col_meta].to_csv("Wang2023.tab.gz",sep="\t")